In [58]:
#import general packages
import numpy as np
from scipy.stats import spearmanr
from colorama import Fore, Style, Back
from pathlib import Path

%load_ext autoreload
%autoreload 2
import exposures_climada as ex
import impact as cimp
import bound_prot_dam as bpd
import n_fct_t_rl_thm_ll as bond_fct
import prem_ibrd as prib
import haz_int_grd as hig
import set_nominal as snom
import functions as fct
import calc_premium as cp
import simulate_bond as sb
import alt_pay_opt as apo
import simulate_multi_cty_bond as smcb


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#choose country using 3-digits numeric country code
country = 308 #Grenada

#define minimum return period to be covered
lower_rp = 25

#define maximum return period to be covered
upper_rp = 50

#define benchmark sharpe ratio
target_sharpe = 0.5

#define the risk free rate
rf_rate = 0.05

## Create Multiple Single-Country Bonds

To create a multi-country bond we first have to create more single-country bonds. We will do this for the carribean state St. Kitts and Nevis using the same setting as for Grenada, so the bond should insure damages with a return period of 25 years or higher and has a capital equal to a damage with a return period of 50 years.

Additionally, we will create a bond for the island state Comoros, located in the Southern Indian basin close to Madagascar. As Comoros is not as heavily affected by tropical cyclones as our chosen Caribean countries we will use higher return periods. The bond we will create is designed to insure damages with a return period of 100 years or larger and comes with a capital suitable to protect from a damage with a return period of 250 years. Consequently, this bond should have a much lower attachment probability as well as expected loss compared to the other bonds.

In [6]:
print(Back.RED + "Grenada")
bond_metrics_gren, returns_gren, premium_dic_gren, nominal_gren, pay_dam_df_gren, es_metrics_gren, int_grid_gren, imp_per_event_flt_gren, imp_admin_evt_flt_gren = bond_fct.sng_cty_bond(country=country, 
                                                                                                                                                                                         prot_share=upper_rp, 
                                                                                                                                                                                         rf_rate=rf_rate, 
                                                                                                                                                                                         target_sharpe=target_sharpe, 
                                                                                                                                                                                         low_to_prot=lower_rp)
print(Back.RED + "St. Kitts and Nevis")
bond_metrics_kitt, returns_kitt, premium_dic_kitt, nominal_kitt, pay_dam_df_kitt, es_metrics_kitt, int_grid_kitt, imp_per_event_flt_kitt, imp_admin_evt_flt_kitt = bond_fct.sng_cty_bond(country=659, 
                                                                                                                                                                                         prot_share=upper_rp, 
                                                                                                                                                                                         rf_rate=rf_rate, 
                                                                                                                                                                                         target_sharpe=target_sharpe, 
                                                                                                                                                                                         low_to_prot=lower_rp)

print(Back.RED + "Comoros")
bond_metrics_com, returns_com, premium_dic_com, nominal_com, pay_dam_df_com, es_metrics_com, int_grid_com, imp_per_event_flt_com, imp_admin_evt_flt_com = bond_fct.sng_cty_bond(country=882, 
                                                                                                                                                                                prot_share=upper_rp, 
                                                                                                                                                                                rf_rate=rf_rate, 
                                                                                                                                                                                target_sharpe=target_sharpe, 
                                                                                                                                                                                low_to_prot=lower_rp)

Grenada
Number of tracks in NA basin: 2692
The principal of the cat bond is: 104716116.948 [USD]
Principal as share of GDP: 0.1
St. Kitts and Nevis
Number of tracks in NA basin: 946
The principal of the cat bond is: 80453906.699 [USD]
Principal as share of GDP: 0.091
Comoros
Number of tracks in SP basin: 1767
The principal of the cat bond is: 128893255.49 [USD]
Principal as share of GDP: 0.148


Now that we have created the bonds we can assess the basics of them. First we see that the subareas for the payout are equal to the islands of the countries except for the largest island of Comoros which is big enough to be divided into two subareas. According to to our algorithm an area should only be divided into subareas when the size of each area can be equal or bigger than 600 km^2. 

Next, much more tropical cyclone events can potentially trigger a payout in St. Kitts and Nevis compared to Comoros.

The damages caused by tropical cyclones in the two countries differ a lot. However, due to the differing chosen return periods, the total needed capital available in the bonds is roughly the same (105 vs 116 million USD). As the GDPs of both countries are similar this corresponds to roughly 10% of each countries GDP.

We will also recreate the cat bond for Grenada which we discussed in the previous notebook.

In [7]:
#print relevant metrics for both single country bonds
print(Back.RED + "Grenada")
print(Style.RESET_ALL)
fct.print_sng_bnd_rel_metr(bond_metrics=bond_metrics_gren, 
                           returns=returns_gren, 
                           premium_dic=premium_dic_gren, 
                           nominal=nominal_gren)
print(Back.RED + "St. Kitts and Nevis")
print(Style.RESET_ALL)
fct.print_sng_bnd_rel_metr(bond_metrics=bond_metrics_kitt, 
                           returns=returns_kitt, 
                           premium_dic=premium_dic_kitt, 
                           nominal=nominal_kitt)
print(Style.RESET_ALL)
print(Back.RED + "Comoros")
print(Style.RESET_ALL)
fct.print_sng_bnd_rel_metr(bond_metrics=bond_metrics_com, 
                           returns=returns_com, 
                           premium_dic=premium_dic_com, 
                           nominal=nominal_com)

Grenada

Expected Loss: 3.4 %
Attachment Probability: 3.8 %
Coverage: 59.0 %
Premium Ibrd: 6.7 %; 7028695.0 USD
Premium Chatoro et al. 9.0 %; 9433437.0 USD
Premium Target Sharpe Ratio 13.6 %; 14200350.0 USD
Standard Deviation of Returns 0.18
St. Kitts and Nevis

Expected Loss: 2.8 %
Attachment Probability: 3.5 %
Coverage: 52.4 %
Premium Ibrd: 6.2 %; 4982138.0 USD
Premium Chatoro et al. 8.3 %; 6653661.0 USD
Premium Target Sharpe Ratio 12.0 %; 9641596.0 USD
Standard Deviation of Returns 0.16

Comoros

Expected Loss: 4.1 %
Attachment Probability: 5.2 %
Coverage: 82.9 %
Premium Ibrd: 7.5 %; 9654944.0 USD
Premium Chatoro et al. 10.1 %; 12985458.0 USD
Premium Target Sharpe Ratio 16.7 %; 21496701.0 USD
Standard Deviation of Returns 0.2


The expected loss for St. Kitts and Nevis is much higher compared to Comoros. This difference is also seen in the price which has to be paid, which is much lower for Comoros. Interestingly, while we see the same pricing pattern for St. Kitts and Nevis compared to Grenada (IBRD is cheapest and meeting the target sharpe ratio the most expensive), Comoros shows a bit of a different pattern. While the IBRD price still is the lowest, the price for the target sharpe ratio comes second. This is because the standard deviation of returns is much lower for Comoros (0.08) compared to St. Kitts and Nevis (0.18), thus the expected returns are allowed to be much lower for Comoros in order to meet the same sharpe ratio. 

The coverage provided to the countries is 52.5% for St. Kitts and Nevis and 43.2% for Comoros.

## Create Multi-Country Bonds

Now we can finally create multi-country bonds. We have 3 different countries which we can be pooled. Lets explore our options and the consequences for the prices. 
The pooling takes place under the constraint that the coverage for the countries has to be exactly the same compared to their individual single-country bond. 

First, we need to create lists of countries we want to pool and then create dictionaries for the capital used in the bond for each country in the pool (nominal) and also for the dataframes containing the year, payout and damage per event (pay_dam_df). In the code chunk below, this is done for all possible pooling combinations.

In the notebook only one example for pooling is showed. However feel free to use the provided functions and dictionaries to create your own country pool. You can also change the settings for the single-country bonds (e.g. what events they should insure) and have a look how this affects the prices, the coverage, and the pooling combinations.

In [8]:
country_1 = 308 #Grenada
country_2 = 659 #St. Kitts and Nevis
country_3 = 882 #Comoros

#Pool 1: Grenada and St. Kitts and Nevis
countries_1 = [country_1, country_2]
nominal_dic_1 = {country_1: nominal_gren,
                 country_2: nominal_kitt}
pay_dam_df_dic_1 = {country_1: pay_dam_df_gren,
                    country_2: pay_dam_df_kitt}

#Pool 2: Grenada and Comoros
countries_2 = [country_1, country_3]
nominal_dic_2 = {country_1: nominal_gren,
                 country_3: nominal_com}
pay_dam_df_dic_2 = {country_1: pay_dam_df_gren,
                    country_3: pay_dam_df_com}

#Pool 3: St. Kitts and Nevis and Comoros
countries_3 = [country_2, country_3]
nominal_dic_3 = {country_2: nominal_kitt,
                 country_3: nominal_com}
pay_dam_df_dic_3 = {country_2: pay_dam_df_kitt,
                    country_3: pay_dam_df_com}

#Pool 4: Grendada, St. Kitts and Nevis, and Comoros
countries_4 = [country_1, country_2, country_3]
nominal_dic_4 = {country_1: nominal_gren,
                 country_2: nominal_kitt,
                 country_3: nominal_com}
pay_dam_df_dic_4 = {country_1: pay_dam_df_gren,
                    country_2: pay_dam_df_kitt,
                    country_3: pay_dam_df_com}

The following will create a multi-county bond insuring Grenada and St. Kitts and Nevis providing both countries exactly the same coverage compared to their indiviudal single-country bonds.

In [75]:
bond_metrics_pool_1, returns_pool_1, tot_coverage_cty_pool_1, premium_dic_pool_1, nominal_pool_1, es_metrics_pool_1, MES_cty_pool_1 = bond_fct.mlt_cty_bond(countries=countries_1,
                                                                                                                                                            pay_dam_df_dic_ps=pay_dam_df_dic_1,
                                                                                                                                                            nominals_dic_ps=nominal_dic_1,
                                                                                                                                                            rf_rate=rf_rate,
                                                                                                                                                            target_sharpe=target_sharpe)



In [76]:
print(Back.RED + 'Pool 1: Grenada + St. Kitts and Nevis')
print(Style.RESET_ALL)
fct.print_mlt_bnd_rel_metr(countries=countries_1, returns=returns_pool_1,
                           premium_dic=premium_dic_pool_1,
                           tot_coverage_cty=tot_coverage_cty_pool_1,
                           nominal=nominal_pool_1)

Pool 1: Grenada + St. Kitts and Nevis

Expected Loss:  3.8 %
Attachment Probability:  8.8 %
Coverage 308: 59.0 %
Coverage 882: 82.9 %
Premium Ibrd:  7.1 %;  16667177.0 USD
Premium Chatoro et al. 9.6 %;  22418895.0 USD
Premium Target Sharpe Ratio 12.0 %;  28118312.0 USD
Standard Deviation Returns 0.1437203298687703


In [77]:
ibrd_sng_pool_1 = premium_dic_gren['ibrd']*nominal_gren + premium_dic_com['ibrd']*nominal_com
regr_sng_pool_1 = premium_dic_gren['regression']*nominal_gren + premium_dic_com['regression']*nominal_com
requ_sng_pool_1 = premium_dic_gren['required']*nominal_gren + premium_dic_com['required']*nominal_com

print(Back.RED + 'Pool 1: Grenada + St. Kitts and Nevis')
print(Style.RESET_ALL)
print('Chatoro et al.: ',  round((1 - (premium_dic_pool_1['regression'] * nominal_pool_1) / regr_sng_pool_1)*100,1),'% - no change in price')
print('IBRD: ',  round((1 - (premium_dic_pool_1['ibrd'] * nominal_pool_1) / ibrd_sng_pool_1)*100,1),'% - no change in price')
print('Target Sharpe Ratio: ',  round((1 - (premium_dic_pool_1['required'] * nominal_pool_1) / requ_sng_pool_1)*100,1),'% cheaper')

Pool 1: Grenada + St. Kitts and Nevis

Chatoro et al.:  0.0 % - no change in price
IBRD:  0.1 % - no change in price
Target Sharpe Ratio:  21.2 % cheaper


There is no change in the price when using the regression model. As expected loss is an additive property and the price is modeled as a linear function of the expected loss it does not matter if we pool the countries or not. Futhermore, there is also no change in price when using the IBRD method. This is not as trivial as for the regression model, but based on the 3D-plot shown during the presentation pooling countries with high losses should not result in any change regarding the price. 

However, the price to meet a specified sharpe ratio changed significantly. Now we can achieve a sharpe ratio of 0.5 by paying 22.5% less premiums annualy.

Feel free to explore more pooling options, or changing the bonds set-ups. There is much more to expolore!